In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os

### Definindo funções comuns

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
def extrair_links(html, lista):
    soup = BeautifulSoup(html, 'html.parser')
    produtos = soup.find_all('div', class_='produto')
    
    for produto in produtos:
        link = produto.find('a', href=True)
        
        desconto_produto = produto.find('div', class_='desconto-produto')
        preco_desconto_element = desconto_produto.find('span', class_='font-weight-bold') if desconto_produto else None
        preco_element = produto.find('div', {'data-target': 'preco-produto'})
        
        preco_com_desconto = preco_element.find('span', class_='font-weight-bold').get_text(strip=True) if preco_element and preco_element.find('span', class_='font-weight-bold') else None
        preco_sem_desconto_tag = preco_element.find('p', class_='text-muted font-sm font-weight-semibold') if preco_element else None
        preco_sem_desconto = preco_sem_desconto_tag.get_text(strip=True) if preco_sem_desconto_tag else None

        resultado = {'URL': 'https://www.farmaciasindependente.com.br' + link['href'], 'Preço sem desconto': preco_sem_desconto, 'Preço com desconto': preco_com_desconto}

        lista.append(resultado)
    return lista

def extrair_infos(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        nome_elemento = soup.find('h1', {'data-target': 'nome_produto'})
        ean_elemento = soup.find('p', class_='text-muted my-2 font-sm')
        marca_elemento = soup.find('p', class_='font-weight-bolder')

        nome = nome_elemento.get_text(strip=True) if nome_elemento else None
        ean = ean_elemento.get_text(strip=True).split('EAN: ')[1] if ean_elemento else None
        marca = marca_elemento.find('a').get_text(strip=True) if marca_elemento else None
        
        #Descrição
        td_elements = soup.find_all('td', class_='tr-pedido w-50')
        descricao_elemento = td_elements[3] if len(td_elements) > 2 else None
        descricao = descricao_elemento.get_text(strip=True) if descricao_elemento else None

        informacoes = {
            'Produto': nome,
            'EAN': ean,
            'Marca': marca,
            'Descrição': descricao
        }
        return informacoes

    except Exception as e:
        print(f"Erro ao processar a URL {url}: {e}")
        return None
    
def process_url(url):
    info = extrair_infos(url)
    if info:
        return info

A primeira aba do código trabalha na extração dos links e preços dos produtos por categorias, e posteriormente acessamos cada link para extrair o restante das informações.

### Extraindo links

In [8]:
categorias = ['medicamentos', 'saude', 'alimentos-e-bebidas', 'higiene-e-cuidados', 'conveniencia', 'beleza-e-perfumaria', 'mamae-e-bebe']

driver = webdriver.Chrome()
links = []
x = 0
for categoria in categorias:
    ref = f"https://www.farmaciasindependente.com.br/categorias/{categoria}"
    driver.get(ref)
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.load.load-body')))
            extrair_links(driver.page_source, links)

            # Encontrar e clicar no botão de próxima página
            next_page_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-pagina="next"]')))
            next_page_button.click()
            time.sleep(2)

        except (StaleElementReferenceException, TimeoutException):
            break

driver.quit()

### Extraindo informações

In [9]:
# Criando df com links e salvando arquivo
df_links = pd.DataFrame(links)
df_links = df_links.drop_duplicates()
df_links.to_csv('df_links.csv', index=False)

In [10]:
# Visitaremos cada link e puxaremos as informações desejadas do html
max_workes = os.cpu_count()

infos = []

with ThreadPoolExecutor(max_workers=max_workes) as executor:
    # Use the map function to apply the process_url function to each URL in parallel
    results = list(tqdm(executor.map(process_url, df_links['URL']), total=len(df_links)))

# Collect the results
infos = list(filter(None, results))

df_infos = pd.DataFrame(infos)
df_infos.to_csv('df_infos.csv')

 41%|████      | 3626/8811 [14:48<17:28,  4.94it/s]  

Erro ao processar a URL https://www.farmaciasindependente.com.br/assert-25mg-cx-7-comp: 404 Client Error: Not Found for url: https://www.farmaciasindependente.com.br/assert-25mg-cx-7-comp


 41%|████      | 3628/8811 [14:48<15:39,  5.51it/s]

Erro ao processar a URL https://www.farmaciasindependente.com.br/anador-gts-10ml: 404 Client Error: Not Found for url: https://www.farmaciasindependente.com.br/anador-gts-10ml


 45%|████▍     | 3934/8811 [15:59<27:24,  2.97it/s]

...

 57%|█████▋    | 4992/8811 [20:17<16:52,  3.77it/s]  

Erro ao processar a URL https://www.farmaciasindependente.com.br/agulha-descartavel-descarpack-30x7mm: list index out of range


 87%|████████▋ | 7702/8811 [31:01<03:11,  5.79it/s]

Erro ao processar a URL https://www.farmaciasindependente.com.br/eximia-temporize-30-capsulas: 404 Client Error: Not Found for url: https://www.farmaciasindependente.com.br/eximia-temporize-30-capsulas


100%|██████████| 8811/8811 [35:11<00:00,  4.17it/s]


### Tratamento de dados para output final

In [13]:
# Algumas linhas retornaram erros para o dataframe de informações no momento da extração, então essas serão removidas do dataframe de links para termos o mesmo número de linhas
links_a_remover = ['https://www.farmaciasindependente.com.br/assert-25mg-cx-7-comp', 'https://www.farmaciasindependente.com.br/anador-gts-10ml', 'https://www.farmaciasindependente.com.br/glimepirida-4mg-30-comprimidos-eurofarma', 'https://www.farmaciasindependente.com.br/agulha-descartavel-descarpack-30x7mm', 'https://www.farmaciasindependente.com.br/eximia-temporize-30-capsulas']
df = df_links[~df_links['URL'].isin(links_a_remover)]
df = df.reset_index(drop=True)

# Juntando os dois dataframes
df_final = pd.concat([df_infos, df.iloc[:, -2:]], axis=1)

In [14]:
# Formatando as colunas de preços e adicionando a % de desconto
df_final['Preço sem desconto'] = df_final['Preço sem desconto'].replace('[^\d,]', '', regex=True).str.replace(',', '.').astype(float)
df_final['Preço com desconto'] = df_final['Preço com desconto'].replace('[^\d,]', '', regex=True).str.replace(',', '.').astype(float)

# Fill NaN values in 'Preço sem desconto' with 'Preço com desconto'
df_final['Preço sem desconto'].fillna(df_final['Preço com desconto'], inplace=True)

# Calculate '% de desconto'
df_final['% de desconto'] = df_final.apply(
    lambda row: "{:.1f}%".format(round((row['Preço sem desconto'] - row['Preço com desconto']) / row['Preço sem desconto'] * 100))
    if pd.notna(row['Preço sem desconto']) and pd.notna(row['Preço com desconto']) and row['Preço sem desconto'] != row['Preço com desconto']
    else None,
    axis=1
)

In [15]:
# Adicionando colunas finais
df_final['Farmacia'] = 'Santo Remedio'
df_final['Região'] = 'Sudeste'
df_final['Cidade'] = 'Sao Paulo'

In [16]:
df_final.to_csv('df_final.csv', index=False)